# PDF RAG System with Vietnamese Language Support

This notebook implements a Retrieval-Augmented Generation (RAG) system for PDF documents with support for Vietnamese language using:
- LangChain for the RAG pipeline
- Qdrant for vector storage
- Google Gemini for embedding and generation
- PyPDF and PyMuPDF for PDF processing

## Install Required Packages

First, let's install all necessary packages:

## Import Dependencies

In [ ]:
# -----------------------------
# 🔑  PLACEHOLDER CONFIG
# -----------------------------
import os

# -- Google Gemini / Gemini 1.5-flash –
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "your_api_key")
# -- Jina AI Embeddings –
JINA_API_KEY = os.getenv("JINA_API_KEY", "your_api_key")
# Put the keys in env so downstream libs pick them up automatically
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["JINA_API_KEY"] = JINA_API_KEY
# -----------------------------

import tempfile
import fitz  # PyMuPDF
from typing import List
import google.generativeai as genai

# LangChain / LLM + Embeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.embeddings import JinaEmbeddings

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_qdrant import Qdrant, QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

genai.configure(api_key=GOOGLE_API_KEY)

# Initialise Jina embeddings up-front so we can reuse them everywhere
text_embeddings = JinaEmbeddings(
    jina_api_key=JINA_API_KEY, model_name="jina-embeddings-v3"
)
#Qdrant


d:\S4V\rag-pipeline-langchain-qdrant\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## PDF Processing Functions

We'll implement enhanced PDF extraction with support for Vietnamese text using PyMuPDF (fitz):

In [3]:
def extract_text_from_pdf(pdf_path: str) -> List[Document]:
    """
    Extract text from PDF with special handling for Vietnamese text.
    
    Args:
        pdf_path: Path to the PDF file
        
    Returns:
        List of Document objects with text content and metadata
    """
    documents = []
    
    try:
        # Open the PDF file using PyMuPDF
        pdf_document = fitz.open(pdf_path)
        
        # Process each page
        for page_num, page in enumerate(pdf_document):
            # Extract text from the page with improved handling for Vietnamese characters
            text = page.get_text("text")
            
            # Skip empty pages
            if not text.strip():
                continue
                
            # Create a Document object with metadata
            doc = Document(
                page_content=text,
                metadata={
                    "source": pdf_path,
                    "page_number": page_num + 1,
                    "total_pages": len(pdf_document)
                }
            )
            documents.append(doc)
            
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
    
    return documents

def split_documents(documents: List[Document], chunk_size: int = 1024
                    , chunk_overlap: int = 200) -> List[Document]:
    """
    Split documents into chunks for better processing.
    
    Args:
        documents: List of Document objects
        chunk_size: Size of each chunk in characters
        chunk_overlap: Overlap between chunks in characters
        
    Returns:
        List of Document objects split into chunks
    """
    # Create a text splitter optimized for Vietnamese
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
        keep_separator=True
    )
    
    # Split the documents
    return text_splitter.split_documents(documents)

## Extract text from scaned PDF

Implement scanned pdf to documents

In [4]:
import tqdm

In [5]:
from __future__ import annotations
from pathlib import Path
from typing import List, Literal, Optional
from pdf2image import convert_from_path
from langchain.docstore.document import Document
import pytesseract
from tqdm.auto import tqdm
import os

# poppler_path = r"D:\poppler\Library\bin"  # Đảm bảo rằng Poppler đã được cài đúng đường dẫn

def extract_scan_pdf(
    pdf_path: str | Path,
    *,
    lang: str = "vie",
    dpi: int = 300,
    poppler_path: Optional[str] = None,
    extra_tess_config: str = "--psm 6",
) -> List[Document]:
    """
    Chuyển PDF được scan thành list[Document] (LangChain).

    Parameters
    ----------
    pdf_path : str | Path
        Đường dẫn PDF.
    lang : str, default ``"vie"``
        Mã ngôn ngữ Tesseract (có thể 'vie', 'eng+vie', …).
    dpi : int, default 300
        Độ phân giải xuất ảnh; cao hơn → OCR chính xác hơn nhưng chậm hơn.
    poppler_path : str | None
        Đường dẫn thư mục chứa binary `pdftoppm` nếu không có trong PATH (Windows).
    extra_tess_config : str
        Tham số cấu hình bổ sung cho Tesseract (ví dụ `--oem 1`, `--psm 4`).

    Returns
    -------
    List[Document]
        Mỗi trang PDF thành một `Document(page_content, metadata)`.
        Metadata gồm `page` (bắt đầu 1) và `source` (tên file).
    """
    pdf_path = Path(pdf_path).expanduser().resolve()
    if not pdf_path.exists():
        raise FileNotFoundError(f"Không tìm thấy file: {pdf_path}")

    # 1) PDF ➜ hình ảnh
    print("Converting PDF to Image")

    poppler_path = r"D:\poppler\Library\bin"
    try:
        images = convert_from_path(
            pdf_path.as_posix(),
            dpi=dpi,
            poppler_path=poppler_path,
        )
    except Exception as e:
        print(f"Error converting PDF to images: {e}")
        return []

    # 2) OCR từng trang
    print("OCR each Pages")
    docs: List[Document] = []
    for idx, img in enumerate(tqdm(images, desc="🔍 OCR pages", unit="page"), start=1):
        try:
            text = pytesseract.image_to_string(
                img, lang=lang, config=extra_tess_config
            )
        except Exception as e:
            print(f"Error OCR page {idx}: {e}")
            text = ""  # Nếu có lỗi OCR, bỏ qua trang này

        docs.append(
            Document(
                page_content=text,
                metadata={"page": idx, "source": pdf_path.name},
            )
        )
    
    return docs


d:\S4V\rag-pipeline-langchain-qdrant\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create Vector Store with Qdrant

In [6]:
from qdrant_client.http.exceptions import UnexpectedResponse
# giả sử bạn đã có `text_embeddings` và `Document`, `Qdrant` được import

def create_vector_store(
    documents: List[Document],
    collection_name: str = "vietnamese_book_pdf_vectors",
    recreate: bool = False,      # ← thêm tuỳ chọn
) -> Qdrant:
    """
    Tạo (hoặc tái sử dụng) vector store Qdrant.
    
    Args:
        documents: Danh sách Document để index
        collection_name: Tên collection trong Qdrant
        recreate: True => xoá & tạo lại; False => dùng collection cũ nếu đã tồn tại
    """
    client = QdrantClient(url="http://localhost:6333")

    # ------------------------------------------------------------------
    # 1) Chuẩn bị collection
    # ------------------------------------------------------------------
    try:
        if recreate:
            # Xoá nếu tồn tại, rồi tạo mới
            client.recreate_collection(
                collection_name=collection_name,
                vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
            )
        else:
            # Chỉ tạo nếu CHƯA có
            if collection_name not in [
                c.name for c in client.get_collections().collections
            ]:
                client.create_collection(
                    collection_name=collection_name,
                    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
                )
    except UnexpectedResponse as e:
        # Bắt lỗi 409 nhưng để các lỗi khác nổi lên
        if getattr(e, "status_code", None) == 409:
            # Collection đã tồn tại & recreate=False → bỏ qua
            pass
        else:
            raise

    # ------------------------------------------------------------------
    # 2) Khởi tạo Vector store và thêm tài liệu
    # ------------------------------------------------------------------
    vector_store = Qdrant(
        client=client,
        collection_name=collection_name,
        embeddings=text_embeddings,
    )
    vector_store.add_documents(documents)
    return vector_store

## Configure RAG Chain with Gemini

In [12]:
# ─── imports ────────────────────────────────────────────────────────────────
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI


# ─── ONE-SHOT QA (vector search + Gemini) ───────────────────────────────────
def answer_question(
    vector_store,  # Qdrant (hoặc bất kỳ VectorStore nào hỗ trợ similarity_search)
    question: str,  # câu hỏi của user
    chat_history: list,  # list[BaseMessage] (HumanMessage / AIMessage)
    k: int = 10,  # số chunk lấy từ vector search
):
    """
    1. vector_store.similarity_search -> lấy k đoạn context
    2. Nhồi context + lịch sử hội thoại vào prompt
    3. Gọi Gemini-flash, trả về câu trả lời & cập nhật chat_history
    """

    # 1️⃣  Lấy context --------------------------------------------------------
    docs = vector_store.similarity_search(question, k=k)
    context = "\n\n".join(d.page_content for d in docs) or "Không có ngữ cảnh."

    # 2️⃣  Xây prompt ---------------------------------------------------------
    system_prompt = (
        "You are a helpful assistant answering from provided context. "
        "If the question is in Vietnamese, answer in Vietnamese with full "
        "diacritics. If the answer is not in the context, say you don't know.\n\n"
        f"Context:\n{context}"
    )

    # Danh sách message: System + lịch sử + câu hỏi mới
    messages = (
        [SystemMessage(system_prompt)] + chat_history + [HumanMessage(content=question)]
    )

    # 3️⃣  Gọi Gemini ---------------------------------------------------------
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-preview-04-17",
        temperature=0.5,
        additional_kwargs={"generation_config": {"top_p": 0.95, "top_k": 40}},
    )
    print(messages)
    ai_msg: AIMessage = llm.invoke(messages)

    # 4️⃣  Cập nhật lịch sử & trả về -----------------------------------------
    chat_history.extend([HumanMessage(content=question), ai_msg])
    return ai_msg.content  # hoặc return ai_msg nếu bạn cần full object


## File Upload and Processing

In [16]:
# ╔════════════════════════════════════════════════════════════════╗
# ║  🚀  Build the index from a local PDF (no widget, no upload)   ║
# ╚════════════════════════════════════════════════════════════════╝
import os, tempfile, pathlib


def build_index_from_local(pdf_path: str):
    """
    Point to any local PDF, then extracts, chunks, embeds and
    prepares the RAG chain.  Globals `vector_store` and `rag_chain`
    are created exactly like before.
    """
    pdf_path = pathlib.Path(pdf_path).expanduser().resolve()
    if not pdf_path.exists():
        raise FileNotFoundError(f"{pdf_path} not found")

    print(f"📄  Processing: {pdf_path.name}")

    docs = extract_text_from_pdf(str(pdf_path))
    if len(docs) == 0:
        docs = extract_scan_pdf(str(pdf_path))
    print(f"- Extracted {len(docs)} pages")
    chunks = split_documents(docs)
    print(f"- Split into {len(chunks)} chunks")

    global vector_store
    vector_store = create_vector_store(chunks)

    print("\n✅  Ready for questions!")


# ▶️  CHANGE THIS TO WHATEVER PDF YOU WANTh
build_index_from_local("SGK_Toan9.pdf")


📄  Processing: SGK_Toan9.pdf
Converting PDF to Image
OCR each Pages


🔍 OCR pages: 100%|██████████| 119/119 [04:34<00:00,  2.30s/page]


- Extracted 119 pages
- Split into 207 chunks

✅  Ready for questions!


## Question Answering Interface

In [18]:
# Initialize chat history for conversation
chat_history = []
# Example usage
# Replace with your actual question
question = "Hãy cho tôi biết hệ thức lượng trong tam giác vuông?"  # "What is the main content of this document?"
answer = answer_question(vector_store, question, chat_history)
print(f"Question: {question}")
print(f"Answer: {answer}")

[SystemMessage(content='You are a helpful assistant answering from provided context. If the question is in Vietnamese, answer in Vietnamese with full diacritics. If the answer is not in the context, say you don\'t know.\n\nContext:\nkhi xuông dôc là 19 kmih.\nSau bài học này, em đã làm được những gì?\n~ Giải thích được một số hệ thức về cạnh và góc trong tam giác vuông (cạnh góc vuông\nbằng cạnh huyền nhàn với sin góc đối hoặc nhàn với côsin góc kể; cạnh góc vuông bằng\ncạnh góc vuông còn lại nhân với tang góc đối hoặc nhân với côtang góc kề).\n~ Giải quyết được một số vấn đề thực tiễn gắn với tỉ số lượng giác của góc nhọn (tính độ dài\nđoan thẳng, độ lớn góc; áp dụng giải tam giác vuông).\n\n` ma N HUẾ TU chế —“-.6ế""\nPhần HIVH H (VÀ. | l) |\nPhần HIYH HỤU VÀ I) LUUNG\n|\n0hương HỆ THỨ LƯỢNG TRŨNG\nIRM B1RC WUDNG\nTrong chương này, các em sẽ tìm hiểu về các tÌ số\nlượng giác của qóc nhọn là sin (sine), côsin (cosine),\ntang (tangent), côtang (cotangent). Các em sẽ học cách\nsử dụng t